In [12]:
import phrase_entity_extraction as ph

In [25]:
sent = """2018 Barnes & Noble. All Rights Reserved.
This email is an advertisement from Barnes & Noble, Inc.
122 Fifth Avenue New York, NY 10011
Attn: Marketing Preferences
Privacy Policy""".lower() 

max_address = ph.return_max_address3(ph.seq, sent)
if sent.upper().strip() == max_address.upper().strip():
    print("ALL GOOD!")
else:
    print()
    print("## ************** NOT THE SAME!!!! *********************##\n")
encoding = ph.encode_from_word_list(ph.w(sent))
print("Max Address: ", max_address, "\n\nWORD ARRAY: ", ph.w(sent), "\n\nENCODING: ", encoding)
print('\n\n{},"{}"'.format(sent.upper(), encoding))


## ************** NOT THE SAME!!!! *********************##

Max Address:  122 FIFTH AVENUE NEW 10011 

WORD ARRAY:  ['2018', 'barnes', '&', 'noble', 'all', 'rights', 'reserved', 'this', 'email', 'is', 'an', 'advertisement', 'from', 'barnes', '&', 'noble', ',', 'inc', '122', 'fifth', 'avenue', 'new', 'york', ',', 'ny', '10011', 'attn:', 'marketing', 'preferences', 'privacy', 'policy'] 

ENCODING:  [['DIGIT', 'ADR_HEAD'], ['ALPHA'], [], ['ALPHA'], ['ALPHA'], ['ALPHA'], ['ALPHA'], ['ALPHA'], ['ALPHA'], ['WAY'], ['ALPHA'], ['ALPHA'], ['ALPHA'], ['ALPHA'], [], ['ALPHA'], ['COMMA'], ['ALPHA'], ['DIGIT', 'ADR_HEAD'], ['ALPHA', 'COMMONST'], ['WORDWAY'], ['ALPHA', 'COMMONST'], ['ALPHA', 'COMMONST'], ['COMMA'], ['ALPHA'], ['DIGIT', 'ADR_HEAD'], ['DELEG'], ['ALPHA'], ['ALPHA'], ['ALPHA'], ['ALPHA']]


2018 BARNES & NOBLE. ALL RIGHTS RESERVED.
THIS EMAIL IS AN ADVERTISEMENT FROM BARNES & NOBLE, INC.
122 FIFTH AVENUE NEW YORK, NY 10011
ATTN: MARKETING PREFERENCES
PRIVACY POLICY,"[['DIGIT', 'ADR_HE

In [26]:
ph.return_best_fit(ph.seq, sent)

[[18, 22, 4, ['ADDRESS'], '122 fifth avenue new'],
 [25, 26, 1, ['SUITE'], '10011'],
 [26, 29, 3, ['ATTN'], 'attn: marketing preferences']]

In [15]:
arr_cands = ph.get_markers(ph.seq, sent, ['ADDRESS', 'POBOX', 'SUITE'])
arr_cands

[[12, 13, 1, ['SUITE'], '2'],
 [14, 15, 1, ['SUITE'], '2018'],
 [22, 23, 1, ['SUITE'], 'u'],
 [23, 24, 1, ['SUITE'], 's'],
 [32, 33, 1, ['SUITE'], 'not'],
 [37, 38, 1, ['SUITE'], '2018'],
 [37, 39, 2, ['ADDRESS'], '2018 michaels'],
 [42, 43, 1, ['SUITE'], '8000'],
 [42, 44, 2, ['ADDRESS'], '8000 bent'],
 [42, 45, 3, ['ADDRESS'], '8000 bent branch'],
 [42, 46, 4, ['ADDRESS'], '8000 bent branch drive'],
 [50, 51, 1, ['SUITE'], '75063']]

In [16]:
def return_best_fit(seq, sent, book_fit=False):
    items_of_interest = ['POBOX', 'ADDRESS', 'ATTN', 'SUITE','_DIR_']
    markers = ph.get_markers(seq, sent, items_of_interest)
    def get_sorted_entity(_markers, entity):
        entities = [arr for arr in _markers if arr[3][0] == entity]
        entities.sort(key=lambda x: int(x[2]))
        return entities
    
    def entitys_overlap(ent1, ent2):
        if ent1[1] <= ent2[0] or ent2[1] <= ent1[0]:
            return False
        else:
            return True
    
    def add_next(markers, best_fit, entity):
        suites = get_sorted_entity(markers, entity)
        idx = len(suites) - 1
        while idx >= 0:
            if not any([entitys_overlap(item, suites[idx]) for item in best_fit]):
                best_fit.append(suites[idx])
                break
            else:
                idx -= 1
        return best_fit
    
    def book_best_fit(arr_domain, markers):
        def is_in_dictionary(markers, start, endplus):
            match_starts = [item for item in markers if item[0] == start]
            match_both = [item for item in match_starts if item[1] == endplus]
            return match_both
        last_length = [-1]*len(arr_domain)
        for i in range(len(arr_domain)):
            if is_in_dictionary(markers, 0, i+1):
                last_length[i] = i + 1
            if last_length[i] == -1:
                for j in range(i):
                    if last_length[j] != -1 and is_in_dictionary(markers, j+1, i+1):
                        last_length[i] = i - j
                        break
        decompositions = []
        if last_length[-1] != -1:
            idx = len(arr_domain) - 1
            while idx >= 0:
                decompositions.append(is_in_dictionary(markers,idx + 1 - last_length[idx], idx + 1)[0])
                idx -= last_length[idx]
            decompositions = decompositions[::-1]
        return decompositions
    
    best_fit = []
    if not book_fit:
        for nugget in items_of_interest:
            best_fit = add_next(markers, best_fit, nugget)
            best_fit.sort(key=lambda x: int(x[0]))
    else:
        best_fit = book_best_fit(ph.w(sent), markers)

    return best_fit

In [17]:
return_best_fit(ph.seq, sent, book_fit=True)

[]

In [18]:
def return_max_address2(seq, sent):
    kinds = ['ADDRESS', 'POBOX', 'SUITE']
    decomposition = ph.decompose_into_dictionary_words(ph.w(sent.lower()), seq, kinds)
    found_tuples = decomposition[3]
    max_address = []
    for kindof, value in found_tuples:
        if kindof and kindof[0] in kinds:
            max_address.append(value)
    return " ".join(max_address).upper()

In [19]:
ph.decompose_into_dictionary_words(ph.w(sent), ph.seq, ['ADDRESS', 'SUITE', 'POBOX'])

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 2, -1, -1, -1, 1, 2, 3, 4, -1, -1, -1, -1, 1]


([-1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  1,
  -1,
  1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  1,
  1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  1,
  -1,
  -1,
  -1,
  -1,
  1,
  2,
  -1,
  -1,
  -1,
  1,
  2,
  3,
  4,
  -1,
  -1,
  -1,
  -1,
  1],
 ['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ['SUITE'],
  '',
  ['SUITE'],
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ['SUITE'],
  ['SUITE'],
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ['SUITE'],
  '',
  '',
  '',
  '',
  ['SUITE'],
  ['ADDRESS'],
  '',
  '',
  '',
  ['SUITE'],
  ['ADDRESS'],
  ['ADDRESS'],
  ['ADDRESS'],
  '',
  '',
  '',
  '',
  ['SUITE']],
 ['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ['SUITE'],
  '',
  ['SUITE'],
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ['SUITE'],
  ['SUITE'],
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ['SUITE'],
  '',
  '',
  '',
  '',
  ['ADDRESS'],
  '',
  '',
  '',
  ['ADDRESS'],
 

In [20]:
def book_best_fit(arr_domain, markers):
    def is_in_dictionary(markers, start, endplus):
        match_starts = [item for item in markers if item[0] == start]
        match_both = [item for item in match_starts if item[1] == endplus]
        return match_both
    last_length = [-1]*len(arr_domain)
    for i in range(len(arr_domain)):
        if is_in_dictionary(markers, 0, i+1):
            last_length[i] = i + 1
        if last_length[i] == -1:
            for j in range(i):
                if last_length[j] != -1 and is_in_dictionary(markers, j+1, i+1):
                    last_length[i] = i - j
                    break
    decompositions = []
    if last_length[-1] != -1:
        idx = len(arr_domain) - 1
        while idx >= 0:
            decompositions.append(is_in_dictionary(markers,idx + 1 - last_length[idx], idx + 1)[0])
            idx -= last_length[idx]
        decompositions = decompositions[::-1]
    return decompositions

book_best_fit(ph.w(sent), arr_cands)

[]

In [21]:
filtered_markers = [marker for marker in arr_cands if not (marker[0] == 0 and marker[2] == 1)]
print(filtered_markers)
arr_cands

[[12, 13, 1, ['SUITE'], '2'], [14, 15, 1, ['SUITE'], '2018'], [22, 23, 1, ['SUITE'], 'u'], [23, 24, 1, ['SUITE'], 's'], [32, 33, 1, ['SUITE'], 'not'], [37, 38, 1, ['SUITE'], '2018'], [37, 39, 2, ['ADDRESS'], '2018 michaels'], [42, 43, 1, ['SUITE'], '8000'], [42, 44, 2, ['ADDRESS'], '8000 bent'], [42, 45, 3, ['ADDRESS'], '8000 bent branch'], [42, 46, 4, ['ADDRESS'], '8000 bent branch drive'], [50, 51, 1, ['SUITE'], '75063']]


[[12, 13, 1, ['SUITE'], '2'],
 [14, 15, 1, ['SUITE'], '2018'],
 [22, 23, 1, ['SUITE'], 'u'],
 [23, 24, 1, ['SUITE'], 's'],
 [32, 33, 1, ['SUITE'], 'not'],
 [37, 38, 1, ['SUITE'], '2018'],
 [37, 39, 2, ['ADDRESS'], '2018 michaels'],
 [42, 43, 1, ['SUITE'], '8000'],
 [42, 44, 2, ['ADDRESS'], '8000 bent'],
 [42, 45, 3, ['ADDRESS'], '8000 bent branch'],
 [42, 46, 4, ['ADDRESS'], '8000 bent branch drive'],
 [50, 51, 1, ['SUITE'], '75063']]